In [27]:
import numpy as np
import json
import os
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from skimage.draw import disk
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, concatenate, Activation, BatchNormalization, Input, Dropout, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Maximum number of cores we expect in any image
MAX_CORES = 256  # Adjust this number based on your dataset


In [2]:
def create_label_array(json_data):
    labels = []
    for item in json_data:
        labels.append([item['x'], item['y'], item['radius']])
    # If there are fewer cores than MAX_CORES, we pad the remaining values with -1
    while len(labels) < MAX_CORES:
        labels.append([-1, -1, -1])  # Padding
    labels_flat = np.array(labels).flatten()
    # Normalize and flatten the labels to be between 0 and 1, except for padding values which remain -1
    for i in range(0, len(labels_flat), 3):
        if labels_flat[i] != -1:
            labels_flat[i] /= 1024
            labels_flat[i+1] /= 1024
            labels_flat[i+2] /= 1024
    return labels_flat

def load_images_and_labels(image_dir, label_dir):
    image_files = [os.path.join(image_dir, file) for file in sorted(os.listdir(image_dir)) if file.endswith('.png')]
    label_files = [os.path.join(label_dir, file) for file in sorted(os.listdir(label_dir)) if file.endswith('.json')]

    images = []
    labels = []

    for image_file, label_file in zip(image_files, label_files):
        # Load image
        image = img_to_array(load_img(image_file, color_mode='rgb'))
        images.append(image / 255.0)  # Normalize the image

        # Load corresponding label
        with open(label_file, 'r') as file:
            json_data = json.load(file)
        label = create_label_array(json_data)
        labels.append(label)

    return np.array(images), np.array(labels)


# Usage
image_dir = './TMA_WSI_Padded_PNGs'
label_dir = './TMA_WSI_Labels_updated'
images, labels = load_images_and_labels(image_dir, label_dir)


In [9]:
labels[0]

array([ 0.67484799,  0.9584839 ,  0.02050781,  0.63689222,  0.95836265,
        0.02050781,  0.51830787,  0.95696212,  0.02050781,  0.47422477,
        0.95246469,  0.02050781,  0.4461997 ,  0.95300336,  0.02050781,
        0.40591982,  0.95212648,  0.02050781,  0.33213435,  0.95209927,
        0.02050781,  0.29367329,  0.94923416,  0.02050781,  0.17910226,
        0.94507225,  0.02050781,  0.2550067 ,  0.94649648,  0.02050781,
        0.21700379,  0.94631954,  0.02050781,  0.14083668,  0.93906828,
        0.02050781,  0.48365068,  0.91360363,  0.02050781,  0.17984267,
        0.90260694,  0.02050781,  0.78911114,  0.88022895,  0.02050781,
        0.67867021,  0.88054992,  0.02050781,  0.64090349,  0.87714357,
        0.02050781,  0.25540466,  0.86533323,  0.02050781,  0.21561345,
        0.86383388,  0.02050781,  0.17871942,  0.85931477,  0.02050781,
        0.05869408,  0.85494121,  0.02050781,  0.86796468,  0.84733176,
        0.02050781,  0.82914161,  0.84566085,  0.02050781,  0.68

In [25]:
def conv_block(input_tensor, num_filters, kernel_size=3, do_batch_norm=True):
    # A conv block consists of two convolutions, each followed by a batch normalization and a relu activation.
    x = Conv2D(num_filters, kernel_size, padding='same', kernel_initializer='he_normal')(input_tensor)
    if do_batch_norm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv2D(num_filters, kernel_size, padding='same', kernel_initializer='he_normal')(x)
    if do_batch_norm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def unet_regression(input_size=(1024, 1024, 3), num_filters=64, depth=4, dropout=0.5, batch_norm=True, max_cores = 256):
    # INPUT LAYER
    inputs = Input(input_size)
    # CONTRACTING PATH
    conv_blocks = []
    x = inputs
    for i in range(depth):
        x = conv_block(x, num_filters * (2**i), do_batch_norm=batch_norm)
        conv_blocks.append(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        if dropout:
            x = Dropout(dropout)(x)

    # BOTTLENECK
    x = conv_block(x, num_filters * (2**(depth)), do_batch_norm=batch_norm)
    
    # EXPANSIVE PATH
    for i in reversed(range(depth)):
        num_filters_exp = num_filters * (2**i)
        x = UpSampling2D(size=(2, 2))(x)
        x = concatenate([x, conv_blocks[i]], axis=3)
        x = conv_block(x, num_filters_exp, do_batch_norm=batch_norm)


    # OLD OUTPUT LAYER
    # output = Conv2D(1, 1, activation='sigmoid')(x)
    # model = Model(inputs=inputs, outputs=output)

    # NEW OUTPUT LAYER
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(dropout)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(dropout)(x)
    # Adjust the number of outputs to be 3 * MAX_CORES
    outputs = Dense(3 * max_cores, activation='linear')(x)  # 3 values for each core: x, y, and radius

    model = Model(inputs=inputs, outputs=outputs)

    return model



In [28]:
model = unet_regression(input_size=(1024, 1024, 3), num_filters=64, depth=4, dropout=0.5, batch_norm=True, max_cores = 256)

In [20]:
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 1024, 1024, 3)]      0         []                            
                                                                                                  
 conv2d_144 (Conv2D)         (None, 1024, 1024, 64)       1792      ['input_9[0][0]']             
                                                                                                  
 batch_normalization_144 (B  (None, 1024, 1024, 64)       256       ['conv2d_144[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_144 (Activation  (None, 1024, 1024, 64)       0         ['batch_normalization_14

In [30]:
import tensorflow as tf
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.losses import MeanSquaredError

# Assuming 'images' and 'labels' are loaded and preprocessed correctly

def custom_loss(y_true, y_pred):
    mask = tf.cast(tf.not_equal(y_true, -1), tf.float32)  # Create a mask for the padding
    loss = MeanSquaredError()(y_true * mask, y_pred * mask)  # Apply the mask element-wise
    return tf.reduce_sum(loss) / tf.reduce_sum(mask)  # Normalize by the number of non-padded entries



# Create the U-Net model (ensure this function is defined correctly)
model = unet_regression(input_size=(1024, 1024, 3), num_filters=32, depth=4, dropout=0.5, batch_norm=True)

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), loss=custom_loss)

# Set up callbacks for learning rate scheduling and checkpointing
checkpoint = ModelCheckpoint('model_checkpoint.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, mode='min', min_lr=1e-6)

# Split your data into training and validation sets
# This is a simple split, consider using sklearn's train_test_split for a random split
val_split = 0.1  # Use 10% of data for validation
num_val_samples = int(val_split * len(images))
train_images, val_images = images[:-num_val_samples], images[-num_val_samples:]
train_labels, val_labels = labels[:-num_val_samples], labels[-num_val_samples:]





In [31]:
# Train the model
history = model.fit(
    train_images, 
    train_labels, 
    validation_data=(val_images, val_labels),
    epochs=5,  # Set the number of epochs
    batch_size=3,  # Set the batch size
    callbacks=[checkpoint, reduce_lr]
)

Epoch 1/5
6/6 [==============================] - ETA: 0s - loss: 1.0364e-04 
Epoch 1: val_loss improved from inf to 0.00031, saving model to model_checkpoint.h5
6/6 [==============================] - 206s 35s/step - loss: 1.0364e-04 - val_loss: 3.1125e-04 - lr: 1.0000e-04
Epoch 2/5


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


6/6 [==============================] - ETA: 0s - loss: 1.0111e-04 
Epoch 2: val_loss did not improve from 0.00031
6/6 [==============================] - 213s 36s/step - loss: 1.0111e-04 - val_loss: 3.1348e-04 - lr: 1.0000e-04
Epoch 3/5
6/6 [==============================] - ETA: 0s - loss: 1.0254e-04 
Epoch 3: val_loss did not improve from 0.00031
6/6 [==============================] - 209s 35s/step - loss: 1.0254e-04 - val_loss: 3.1346e-04 - lr: 1.0000e-04
Epoch 4/5
6/6 [==============================] - ETA: 0s - loss: 1.0148e-04 
Epoch 4: val_loss did not improve from 0.00031
6/6 [==============================] - 214s 36s/step - loss: 1.0148e-04 - val_loss: 3.1378e-04 - lr: 1.0000e-04
Epoch 5/5
6/6 [==============================] - ETA: 0s - loss: 1.0059e-04 
Epoch 5: val_loss did not improve from 0.00031
6/6 [==============================] - 219s 37s/step - loss: 1.0059e-04 - val_loss: 3.1330e-04 - lr: 1.0000e-04


In [32]:
model.save("Coordinate_model.h5")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
